# **Laser-Induced Breakdown Spectroscopy Data Analysis Notebook**

### Please save yourself a copy of this notebook, rather than editing the class copy. Go to the top left corner under "File", and click "Save a Copy in Drive". A new copy of the notebook will open in another tab--rename the notebook and you're ready to start!


___


This code is meant to help merge LIBS Excel files into an easier to read format and can help in the automatic determination of the elements present in a sample. The code is designed to be excutable without requiring a background in Python. It is expected that you have completed the corresponding LIBS prelab and have read the LIBS protocol before using this tool. If you have any questions, please let Markace Rainey know at mrainey7@gatech.edu.

First, let's import some common Python packages that are already included in the Google Colab environment. Packages are just collections of prewritten code that we can reuse in many different contexts. For example, we will use the plotly package to produce interactive plots!

In [ ]:
# REQUIRED
# IMPORTING COMMON PACKAGES

import pandas as pd
import numpy as np
import statistics
import plotly.express as px
import time
import plotly.graph_objects as go

Now, let's check whether the Python package Astroquery is installed. If it is, we'll just import it. It not, we'll install it, then import it.
Astroquery is a package that allows us to call information directly from the NIST Atomic Lines Database.

In [ ]:
# REQUIRED
# IMPORTING ASTROQUERY

import pip

def import_or_install(package):
    try:
        __import__(package)
        print("Astroquery imported successfully!")
    except ImportError:
        print("Astroquery not installed. Attempting installation.")
        pip.main(['install', package])
        print("Astroquery imported successfully!")

import_or_install('astroquery')

from astroquery.nist import Nist
import astropy.units as u

The following coding block shows how Astroquery works.
First, we have to specify lower and upper wavelength bounds. The example below searches between 0 nm and 10000 nm; note that the 'u.nm' portion refers to us importing the definition of a nanometer from the units module above.
Next, we have to specify which atom type we are interested in. The example below retrieves the lines for 'Cu I', which represents neutral copper. The remaining query terms are not important for our analysis.

In [ ]:
# OPTIONAL
# PRACTICING ASTROQUERY SEARCHES, PART 1


table = Nist.query(minwav=0*u.nm, maxwav=10000*u.nm, linename='Cu I')
print(table)


Notice that the table is cutting off part of the information and that it includes some columns we might not be interested in. The section below will clean up the table for us, including the removal of lines without relative intensity data. By default, the table is ranked by intensity value, but we can make the table interactive by clicking the magic wand button that says, "Convert this dataframe into an interactive table".

In [ ]:
# OPTIONAL
# CREATING STREAMLINED TABLE

import astropy.units as u

min_value = 200  # replace with your actual minimum value
max_value = 850  # replace with your actual maximum value

# Assuming 'table' is a DataFrame you defined earlier from "Cu I"
datatable_version = table.to_pandas()

# Create a clean copy of the data to avoid SettingWithCopyWarning
datatable_version_clean = datatable_version.dropna(subset=["Rel."]).copy()

# Extracting digits and handling missing values
datatable_version_clean['Intensity'] = [''.join(filter(str.isdigit, str(i))) for i in datatable_version_clean["Rel."]]
datatable_version_clean['Intensity'].replace('', np.nan, inplace=True)
datatable_version_clean.dropna(subset=["Intensity"], inplace=True)

# Convert 'Intensity' to integers
datatable_version_clean['Intensity'] = datatable_version_clean['Intensity'].astype(int)

# Sorting and dropping unnecessary columns
datatable_version_clean.sort_values(by=['Intensity'], ascending=False, inplace=True)
datatable_version_clean.drop(columns=["Aki", "Ei           Ek", "Ritz", "fik", "Transition", "Acc.", "Type", "TP"], inplace=True)

# Convert 'Observed' column to numeric type (float)
datatable_version_clean['Observed'] = pd.to_numeric(datatable_version_clean['Observed'], errors='coerce')

# Now apply the filter with numerical comparison
filtered_table = datatable_version_clean[(datatable_version_clean['Observed'] > min_value) & (datatable_version_clean['Observed'] < max_value)]

# Display the filtered table
display(filtered_table)



Now that we have imported the required packages and are familiar with Astroquery search process, let's switch to looking at our data. First, we need to import our data into Google Colab from our computer. The next coding block will import an Excel file of our choosing; it will then appear in the Files tab that can be viewed by clicking the folder icon on the left-hand panel. Note: There is an occassional error with loading large files, try to rerun the cell a couple of times and it typically works if you are using Google Chrome. If it continues to fail, work with a partner and email Markace (mrainey7@gatech.edu).

In [ ]:
# REQUIRED
# UPLOADING EXCEL DATA

from google.colab import files

uploaded = files.upload()

for initial_file_name in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=initial_file_name, length=len(uploaded[initial_file_name])))

Our LIBS instrument is designed to measure emitted light with a high spectral resolution within the timescale of a laser pulse. To accomplish this goal, we split the emission light into 8 separate spectrometers that each measure a different range of the electromagnetic spectrum in parallel. Unfortunately, the Avasoft software used in our lab exports the data for each of these spectrometers as separate sheets in an Excel workbook. To save you time, the code block below will merge the values on each sheet, then save a new copy of the Excel workbook under the files tab on the left-hand panel. The new Excel workbook will have the same name as your input file, but with "_merged" added to the end. You can view this new document by visting the files tab on the lefthand panel (folder icon), and you can download this merged version for future use.

In [ ]:
# REQUIRED
# CREATING MERGED EXCEL FILE

xl = pd.ExcelFile(initial_file_name)
print("The name of the input file was:", initial_file_name)
if len(xl.sheet_names) > 1:
  print("There are", len(xl.sheet_names), "sheets in the uploaded Excel file. We will merge the sheets for you.")
  full_wavelength = []
  full_intensity = []
  for i in xl.sheet_names:
      full_data = xl.parse(i)
      full_data.reset_index()
      wavelengths = full_data.iloc[:, 0]
      intensities = full_data.iloc[:, 1]
      list_wave = list(wavelengths[range(7,len(wavelengths))])
      list_intensity = list(intensities[range(7,len(intensities))])
      full_wavelength.extend(list_wave)
      full_intensity.extend(list_intensity)
  new_book = pd.DataFrame({'Wavelength (nm)':full_wavelength,'Intensity (au)':full_intensity})

elif len(xl.sheet_names) == 1:
  new_book = pd.read_excel(xl)
  print("The uploaded file appears to already be a single sheet. Only the column titles have been updated.")

final_file_name = initial_file_name[:len(initial_file_name)-5] + '_merged.xlsx'
print("The name of the output file is:", final_file_name)

new_book.to_excel(final_file_name)

Now, let's take a look at the full spectrum for our data. The plot you generate below is fully interactive. There are a list of tools and plot saving options in the toolbar in the top right of the graph. You can save an image of this graph by clicking the camera icon on the plotly toolbar. Pro tip: If you accidently zoom in, double clicking on the graph will reset the axes.

In [ ]:
# REQUIRED
# PLOTTING LIBS SPECTRUM

df = new_book
fig = px.line(x=df["Wavelength (nm)"], y=df["Intensity (au)"])
fig.update_layout(xaxis=dict(title='Wavelength (nm)', showgrid=False),
              yaxis=dict(title='Intensity (Counts)'))
fig.update_layout(template = 'none', height = 500, width = 1500)
fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
config = {'displayModeBar': True}
fig.show(config=config)

### Click the coding block following this text, then come back and read this info. The code takes a couple of minutes!
---

When your spectrum was collected, you presumably adjusted the experimental parameters to alleviate baseline noise and improve the signal. However, baseline noise is ever-present, and the baseline may vary dramatically between different spectrometers. This variation can make the automatic assignment of peaks difficult since a genuine peak in one location might be lower than the baseline in another. To facilitate our analysis, we must apply baseline correction.

**Approach**

In our approach, we will use median baseline correction. We'll create boxes centered on each data point, including the 40 closest neighbors on both the left and the right. We will find the median of those 81 points and subtract it from the center point. We'll repeat this process for each point, and any resulting intensity values less than zero will be set to zero.

**Reasoning**

Peaks in a spectrum are relatively rare. If you select 81 neighboring points, most of those points will likely represent the baseline. Therefore, the median of those 81 points is a good estimation of the baseline at that location. By removing the median of each box, we should be effectively eliminating the baseline fluctuations!

**Complication**

Sometimes, merely subtracting the median of the box may not be enough to remove the baseline variations. In those instances, we can subtract higher multiples of the median until the baseline is satisfactorily corrected. The graph below includes a slider that illustrates what happens when you subtract different multiples of the median. Notice that increasing the multiple will lead to more correction but may result in overcorrection. Undercorrection might cause us to mistakenly match peaks to elements not present in our sample, while overcorrection could lead us to overlook elements present in low abundance. Play with the slider below to explore the effects of different correction levels (note: the median factor is a multiple of the median we are subtracting, so adjusting it changes the degree of correction).

In [ ]:
# OPTIONAL
# EXAMINE BASELINE CORRECTION

# Helper function to apply baseline correction
def correct_intensity(df, step, window_size=40):
    corrected_intensity = [
        max(intensity - statistics.median(df["Intensity (au)"][max(i-window_size, 0):min(i+window_size, len(df))]) * step, 0)
        for i, intensity in enumerate(df["Intensity (au)"])]
    return corrected_intensity

# Create figure
fig = go.Figure()
step_width = 0.1
window_size = 40

# Add traces, one for each slider step
for step in np.arange(0, 2, step_width):
    corrected_intensity = correct_intensity(df, step, window_size)
    fig.add_trace(
        go.Scatter(
            visible=False,
            line=dict(color="#00CED1", width=2),
            name="Median factor = " + str(step),
            x=df["Wavelength (nm)"],
            y=corrected_intensity))

# Make the first trace visible
fig.data[0].visible = True

# Create and add slider
steps = [dict(
    method="update",
    args=[{"visible": [False] * len(fig.data)},
          {"title": "Median factor value: " + str(round(i*step_width, 2))}],
) for i in range(len(fig.data))]

for i in range(len(fig.data)):
    steps[i]['args'][0]['visible'][i] = True

fig.update_layout(
    sliders=[dict(active=0, currentvalue={"prefix": "Median Factor: "}, pad={"t": 50}, steps=steps)],
    template='none', height=500, width=1500,
    xaxis=dict(title='Wavelength (nm)', showgrid=False, showline=True, linewidth=1, linecolor='black', mirror=True),
    yaxis=dict(title='Corrected Intensity (Counts)', showgrid=False, showline=True, linewidth=1, linecolor='black', mirror=True)
)

config = {'displayModeBar': True}
fig.show(config=config)


Once you have found your desired level of correction, enter the corresponding median factor into the cell below. This code block will plot your original raw data and the new baseline corrected data. Double check that all of your peaks of interested are still there! If you think you are missing important peaks, play with the median_factor parameter until you are happy.

In [ ]:
# REQUIRED
# COMPARING BASELINE-CORRECTED AND RAW SPECTRA

####################
median_factor = 1.3 # Change the median factor! 1.3 is the default, but you should select your desired value from the analysis above.
####################

df = new_book
corrected_intensity = []
median_val = statistics.median(df["Intensity (au)"])
for i in range(40, len(df["Intensity (au)"])-40):
    w = df["Intensity (au)"][i]-statistics.median(df["Intensity (au)"][i-40:i+40])*median_factor
    w = w
    if w<0:
        w=0
    corrected_intensity.append(w)

fig = go.Figure()
fig.add_trace(go.Scatter(x=df["Wavelength (nm)"], y=df["Intensity (au)"],  name="Raw Data"))
fig.add_trace(go.Scatter(x=df["Wavelength (nm)"][range(40, len(df["Intensity (au)"])-40)], y=corrected_intensity,  name="Baseline Corrected"))



fig.update_layout(template = 'none')
fig.update_layout(xaxis=dict(title='Wavelength (nm)', showgrid=False),
              yaxis=dict(title='Raw/Corrected Intensity (Counts)')
)

fig.update_layout(template = 'none', height = 500, width = 1500)
fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)


config = {'displayModeBar': True}
fig.show(config=config)

Now that our data is ready to be analyzed, we need to set up our NIST searches. Unfortunately, our wavelength measurements are never perfect. We are using eight separate linear diode arrays (LDA) to detect these peaks; LDAs consist of diodes lined up in a strip, and each diode is aligned to recieve a certain band of wavelengths. Any wavelength that falls into that band is assigned the same wavelength. For example, if we record "693.8802 nm", that means we had a peak somewhere within 693.8802 nm ± 0.06 nm. That error associated with our wavelength measurement is a description of our spectral resolution. In the cell below, are are ploting the distances between the wavelength measurments our instrument records (wavelength [n+1] - wavelength [n]) as a function of wavelength. Notice that we are able to measure closer peaks in some regions of the spectrum than in others. Look over the graph and describe any trends you see with your lab partners. What is the worst our error can ever be, and what wavelength does it correspond to?

In [ ]:
# OPTIONAL
# PLOTTING SPECTRAL (WAVELENGTH) RESOLUTION AS A FUNCTION OF WAVELENGTH

import plotly.graph_objects as go


resolution_values = [df["Wavelength (nm)"][i+1] - df["Wavelength (nm)"][i] for i in range(len(df["Wavelength (nm)"]) - 1)]

fig = go.Figure()
fig.add_trace(go.Scatter(x=df["Wavelength (nm)"][:-1], y=resolution_values, name="Resolution", line=dict(color='black')))

# Define spectrometer sections and colors
spectrometer_sections = [
    (200, 250),
    (250, 300),
    (300, 330),
    (330, 390),
    (390, 470),
    (470, 590),
    (590, 690),
    (690, 850)
]

colors = [
    '#8B0000',
    '#9400D3',
    '#4B0082',
    '#0000FF',
    '#00FF00',
    '#FFFF00',
    '#FF7F00',
    '#FF0000'
]

# Add semi-transparent rectangles for spectrometer sections
for idx, ((start, end), color) in enumerate(zip(spectrometer_sections, colors)):
    fig.add_shape(type='rect',
                  xref='x', yref='y',
                  x0=start, y0=min(resolution_values),
                  x1=end, y1=max(resolution_values),
                  fillcolor=color,
                  opacity=0.25, # Increase opacity
                  layer="below",
                  line=dict(width=0)) # Make line transparent

    # Add a dummy scatter trace for the legend entry
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', line=dict(color=color), name=f"Spectrometer #{idx + 1}"))

fig.update_layout(
    template='none',
    height=500,
    width=800,
    xaxis=dict(title='Wavelength (nm)', showgrid=False, gridcolor='black'),
    yaxis=dict(title='Resolution (Δ nm)', showgrid=False, gridcolor='black')
)

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
config = {'displayModeBar': True}
fig.show(config=config)

print("Minimum wavelength recorded:", min(df["Wavelength (nm)"]))
print("Maximum wavelength recorded:", max(df["Wavelength (nm)"]))
print("Worst spectral resolution:", max(resolution_values))


To keep it simple, let's use the worst resolution for all of our searches. Now, we have a spectrum ready to process and an uncertainty to use for matching peaks. Below is a function definition--it doesn't have an output, but it allows us to determine which elements are present. It uses the Astroquery process from earlier to search for peaks matching an element.

We give the function the following things:

**input_elements**: a list of elements in the form ["H I", "Ne I"...]; these are the elements the function will try to match.

**df**: the baseline corrected data from earlier, in the form of a dataframe.

**top_count**: the number of lines to try to match; for example, a top_count of 20 would pull the 20 highest intensity lines for an element and look for them.

**resolution**: this is the number we determined above which gives a some wiggle room for the wavelength matches.

**threshold**: the number of lines we must match to consider an element to be present.

**min_wl**: the minimum wavelength for our search; our instrument measures between 200 and 850 nm, so this value stays at 200 nm.

**max_wl**: the minimum wavelength for our search; our instrument measures between 200 and 850 nm, so this value stays at 850 nm.

Then the function returns a list of which elements were in the sample, a list of the corresponding peaks that were matched, and a list of peaks that were not found.

In [ ]:
# REQUIRED
# CREATING AN ELEMENT SEARCH FUNCTION

from astropy import units as u
import statistics
import numpy as np


def find_elements(input_elements, df, top_count=20, resolution=0.06, threshold=5, min_wl=200, max_wl=850):

    # Making containers for matching wavelengths and their corresponding elements and intensities.
    ref_wavelength_matches_lists = []
    ref_intensity_matches_lists = []
    ref_element_matches_lists = []
    exp_wavelength_matches_lists = []
    exp_intensity_matches_lists = []


    # Iterate through each element in input_elements
    for element in input_elements:

        # Filtering the input based on intensity
        filtered_input = df[df["Intensity (au)"] > 1]
        filtered_wavelengths = filtered_input["Wavelength (nm)"].values

        # Querying the NIST database for information about the element
        table = Nist.query(minwav= min_wl * u.nm, maxwav = max_wl * u.nm, linename=element)

        # Data cleaning and preparation
        datatable_version = table.to_pandas()
        datatable_version_clean = datatable_version.dropna(subset=["Rel."])
        datatable_version_clean['Intensity'] = [''.join(filter(str.isdigit, str(i))) for i in datatable_version_clean["Rel."]]
        datatable_version_clean['Intensity'].replace('', np.nan, inplace=True)
        datatable_version_clean.dropna(subset=["Intensity"], inplace=True)
        datatable_version_clean['Intensity'] = datatable_version_clean['Intensity'].astype(int)
        final_table = datatable_version_clean[datatable_version_clean["Intensity"] > statistics.median(datatable_version_clean["Intensity"])]
        datatable_version_clean.sort_values(by=['Intensity'], ascending=False, inplace=True)


        datatable_version_clean['Observed'] = pd.to_numeric(datatable_version_clean['Observed'], errors='coerce')

        datatable_version_clean = datatable_version_clean[(datatable_version_clean['Observed'] > min_wl) & (datatable_version_clean['Observed'] < max_wl)]

        # Initializing lists for results
        ref_present = []
        ref_int_present = []
        exp_present = []
        potentially_missing = []

        # Pre-calculation outside the loop
        top_observed = datatable_version_clean.head(top_count)['Observed'].tolist()
        top_rel = datatable_version_clean.head(top_count)['Rel.'].tolist()

        # Searching for lines within the resolution range
        for line_number in range(len(top_observed)):
            i = top_observed[line_number]
            k = top_rel[line_number]

            # Using NumPy to find wavelengths within the resolution range
            within_range = np.abs(filtered_wavelengths - i) < resolution

            if np.any(within_range):
                real_wavelength = filtered_wavelengths[within_range][0]
                ref_present.append(i)
                exp_present.append(real_wavelength)
                ref_int_present.append(k)
            else:
                potentially_missing.append(i)

        # Storing the results if they meet or are above the threshold
        if len(ref_present) >= threshold:
            ref_wavelength_matches_lists.append(ref_present)
            exp_wavelength_matches_lists.append(exp_present)
            ref_element_matches_lists.append(len(ref_present) * [element])
            ref_intensity_matches_lists.append(ref_int_present)

            print("There are", len(ref_present), "of the top", top_count, "lines present for", element)
            print("The following lines are present:", ref_present)
            print("Double check if the following are present:", potentially_missing)
            print("_________________________________________________________________________________________________________________________", '\n')

    return ref_wavelength_matches_lists, exp_wavelength_matches_lists, ref_element_matches_lists, ref_intensity_matches_lists

print("Function loaded!")


First, let's look at an example for why matching a single line is **never** enough to say a metal is present in our sample. None of the samples we assign students in 3216 lab should contain iron, but iron often pops up as an option when we look up peaks from our spectrum.

A major concept for element determination is that if an element is present, most of its highest intensity lines should appear! Below, we first see if any of the top 10 iron lines are in our sample. Usually, this answer is no, none of the most intense iron lines are in our sample. That tells us our sample isn't iron.

Then we search if any of our lines match any of the thousands of iron lines possible. Usually, this leads to many matches.

Discuss with your partner--if the first search shows iron isn't present, why does the second search have so many matches?

In [ ]:
# OPTIONAL
# ILLUSTRATING FALSE DISCOVERY

# Are the top 10 lines of iron in our sample?
df1 = pd.DataFrame()
df1["Wavelength (nm)"]=df["Wavelength (nm)"][range(40, len(df["Intensity (au)"])-40)]
df1["Intensity (au)"]=corrected_intensity
pd.options.mode.chained_assignment = None  # default='warn'
ref_wavelength_matches_lists, exp_wavelength_matches_lists, ref_element_matches_lists, ref_intensity_matches_lists = find_elements(
    ['Fe I'],
    df1,
    top_count = 10,
    resolution = 0.060437395351755185,
    threshold = 0,
    min_wl = 200,
    max_wl = 850)
if len(ref_wavelength_matches_lists)==0:
  print("No lines matched Fe I when we search for the top 10 Fe I lines.")
else:
  print(len(ref_wavelength_matches_lists[0]), "line(s) matched Fe I when we compare to the top 10 Fe I lines.")

print('\n')

# Do any of our lines match any iron lines?
df1 = pd.DataFrame()
df1["Wavelength (nm)"]=df["Wavelength (nm)"][range(40, len(df["Intensity (au)"])-40)]
df1["Intensity (au)"]=corrected_intensity
pd.options.mode.chained_assignment = None  # default='warn'
ref_wavelength_matches_lists, exp_wavelength_matches_lists, ref_element_matches_lists, ref_intensity_matches_lists = find_elements(
    ['Fe I'],
    df1,
    top_count = 6278,
    resolution = 0.060437395351755185,
    threshold = 1,
    min_wl = 200,
    max_wl = 850)

Now that we know how top_count can affect our element matching, let's determine which lines are present in our sample. First, let's be restrictive. We can scan all elements in the NIST website and we can tentatively say the element is present if at least 3 of the top 5 lines are in our spectrum. Run the code below and discuss with your partner whether the elements matched make sense. Remember, you are shooting a metal inside of a chamber filled with a gas. Also keep in mind that some elements may be matched by chance, so be cautious if you find strange answers.

In [ ]:
# REQUIRED
# ELEMENT-MATCHING WITH STRICT SETTINGS FOR ALL ELEMENTS

# Create a list of elements to search for
potential_elements = ["H I", "He I", "Li I", "Be I", "B I", "C I", "N I", "O I", "F I", "Ne I", "Na I", "Mg I", "Al I", "Si I", "P I", "S I", "Cl I", "Ar I","K I", "Ca I", "Sc I", "Ti I", "V I", "Cr I", "Mn I", "Fe I", "Co I", "Ni I", "Cu I", "Zn I", "Ga I", "Ge I", "As I", "Se I", "Br I", "Kr I", "Rb I", "Sr I", "Y I", "Nb I", "Mo I", "Tc I", "Ru I", "Rh I", "Pd I", "Ag I", "Cd I", "In I", "Sn I", "Sb I", "Te I", "I I", "Xe I", "Cs I", "Ba I", "Hf I", "Ta I", "W I", "Re I", "Os I", "Ir I", "Pt I", "Au I", "Hg I", "Tl I", "Pb I", "Li II", "Be II", "B II", "C II", "N II", "O II", "F II", "Ne II", "Na II", "Mg II", "Al II", "Si II", "P II", "S II", "Cl II", "Ar II", "Ca II", "Sc II", "Ti II", "V II", "Cr II", "Mn II", "Fe II", "Co II", "Ni II", "Cu II", "Zn II", "Ga II", "Ge II", "As II", "Se II", "Br II", "Kr II", "Rb II", "Sr II", "Y II", "Zr II", "Nb II", "Mo II", "Tc II", "Ru II", "Rh II", "Ag II", "Cd II", "In II", "Sn II", "Sb II", "Te II", "I II", "Xe II", "Cs II", "Ba II", "Hf II", "Ta II", "W II", "Re II", "Os II", "Pt II", "Au II", "Hg II", "Tl II", "Pb II"]


# Create a new DataFrame using a range of rows from the original DataFrame.
# Limiting the range by excluding the first and last 40 points as the baseline correction algorithm
# cannot be applied to these points. 'corrected_intensity' contains the signal intensities that were baseline corrected.
df1 = pd.DataFrame({
    "Wavelength (nm)": df["Wavelength (nm)"][40:len(df["Intensity (au)"])-40],
    "Intensity (au)": corrected_intensity
})

# Suppress warnings related to chained assignment
pd.options.mode.chained_assignment = None

# Call find_elements function with specific parameters:
# top_count: Number of top lines to consider
# resolution: Tolerance range for matching
# threshold: Minimum number of lines to confirm an element
# min_wl and max_wl: Wavelength range
ref_wavelength_matches_lists, exp_wavelength_matches_lists, ref_element_matches_lists, ref_intensity_matches_lists = find_elements(
    potential_elements,
    df1,
    top_count=3,
    resolution=0.12437395351755185,
    threshold=3,
    min_wl=200,
    max_wl=850
)


# Extracting and printing the preliminary list of elements found
preliminary_elements = [element[0] for element in ref_element_matches_lists]
print("Preliminary elements found:", preliminary_elements)


The code block above helps narrow down the potential elements in our sample. Given the conservative approach of our initial search, we will now conduct a refined search. This next search will explore a larger set of lines, specifically the top 25, for the preliminary elements identified.

When populating the list for the refined search:

Ensure both neutral and ionized forms of the elements are included. For instance, if 'Cu I' was identified, add both 'Cu I' and 'Cu II' to the search.
If any of the identified elements seem unlikely or implausible, consider omitting them from this refined search.

In [ ]:
# REQUIRED
# ELEMENT-MATCHING WITH RELAXED SETTINGS FOR SPECIFIC ELEMENTS

# List of specific elements for a more comprehensive search
specific_elements = ['Ar I', 'Cu I', 'Cu II', "Ca I", "Ca II", "O I", "N I", "Zn I", "Zn II"]

ref_wavelength_matches_lists, exp_wavelength_matches_lists, ref_element_matches_lists, ref_intensity_matches_lists = find_elements(
    specific_elements,
    df1,
    top_count=25,
    resolution=0.120437395351755185,
    threshold=2
)

The output above is the primary component of our post lab assignment. It helps describe which elements are present in our sample and which emission lines support our conclusions. Unfortunately, it's not very pretty, and wouldn't export to Excel nicely. Also, the Recording Your Results section of the protocol requests these results in a very specific format.

*In tabular form, compare the number, wavelength and intensities of expected spectral lines for each element present in your sample with the observed spectral lines.*

We can automate the production of these tables too! We can grab the list of the lines from NIST that were found (Reference Wavelength and Reference Relative Intensity), the element those lines corresponded to (Elemental Assignment), and which peaks in our spectrum they matched (Measured Wavelength and Measured Intensity). First, let's make individual tables for each element matched.

In [ ]:
# REQUIRED
# CREATING REPORT-WORTHY TABLES BY ELEMENT

# Extract raw intensities for matched wavelengths
raw_intensities_lists = []
for element_set in exp_wavelength_matches_lists:
    raw_intensities = [df["Intensity (au)"][df["Wavelength (nm)"] == wavelength].iloc[0] for wavelength in element_set]
    raw_intensities_lists.append(raw_intensities)

# Create and display tables for each set of matched elements
for set_number in range(len(exp_wavelength_matches_lists)):
    element_dictionary = {
        "Peak Wavelength, Measured (nm)": exp_wavelength_matches_lists[set_number],
        "Intensity, Measured (Counts)": raw_intensities_lists[set_number],
        "Elemental Assignment": ref_element_matches_lists[set_number],
        "Peak Wavelength, Reference (nm)": ref_wavelength_matches_lists[set_number],
        "Relative Intensity, Reference (Counts)": ref_intensity_matches_lists[set_number]
    }
    # Create a DataFrame and sort by Elemental Assignment and Intensity, then display
    element_df = pd.DataFrame(element_dictionary)
    display(element_df.sort_values(['Elemental Assignment', 'Intensity, Measured (Counts)'], ascending=[True, False]))


Now, we can merge the individual tables into one large table consisiting of all of the lines found for the elements investigated. Like the eariler tables, this table could be converted into an interactive table with the wizard wand.

In [ ]:
# REQUIRED
# CREATING REPORT-WORTHY TABLES WITH ALL ELEMENTS COMBINED

# List to hold individual tables
tables = []

# Loop through each set of matched elements to create individual tables
for set_number in range(len(exp_wavelength_matches_lists)):
    element_dictionary = {
        "Peak Wavelength, Measured (nm)": exp_wavelength_matches_lists[set_number],
        "Intensity, Measured (Counts)": raw_intensities_lists[set_number],
        "Elemental Assignment": ref_element_matches_lists[set_number],
        "Peak Wavelength, Reference (nm)": ref_wavelength_matches_lists[set_number],
        "Relative Intensity, Reference (Counts)": ref_intensity_matches_lists[set_number]
    }
    element_df = pd.DataFrame(element_dictionary)
    tables.append(element_df)

# Concatenate all the individual tables
combined_table = pd.concat(tables)

# Set option for displaying a large number of rows
pd.set_option('display.max_rows', 500)

# Sort and display the combined table
display(combined_table.sort_values(['Elemental Assignment', 'Intensity, Measured (Counts)'], ascending=[True, False]))


The **Reporting Your Results** section of the LIBS protocol requires you to include two things for each of the samples you analyzed with the LIBS setup; (1) a picture of your spectrum, and (2) a table showing at least five lines matching each element you claim to be in your sample. These last coding blocks simplify the large table from the earlier, showing only the top lines for each element. By default, it shows the five most intense lines matched for each element, but this can be changed with the top_lines_count parameter. It also saves the simplifed table as a downloadable Excel file, which has the same name as the file you uploaded with the suffix "_elemental_analysis_summary". It will appear in the Files tab that can be viewed by clicking the folder icon on the left-hand panel, and you can download it by clicking the three vertical dots. You can then copy and paste the table from the Excel file into your Microsoft Word document.

In [ ]:
# REQUIRED
# PRODUCING SIMPLIFIED TABLE FOR REPORT

###################
top_lines_count = 25 # To change the number of lines per each element, change this number.
###################

# Helper function to get top elements based on intensity
def get_top_elements(elements, count):
    sorted_elements = sorted(zip(elements['exp_wavelengths'], elements['raw_intensities'], elements['ref_elements'], elements['ref_wavelengths'], elements['ref_intensities']), reverse=True)
    return [list(x) for x in zip(*sorted_elements[:count])]

# Helper function to create DataFrame from lists
def create_dataframe(exp_wavelengths, raw_intensities, ref_elements, ref_wavelengths, ref_intensities):
    return pd.DataFrame({
        "Peak Wavelength, Measured (nm)": exp_wavelengths,
        "Intensity, Measured (Counts)": raw_intensities,
        "Elemental Assignment": ref_elements,
        "Peak Wavelength, Reference (nm)": ref_wavelengths,
        "Relative Intensity, Reference (Counts)": ref_intensities
    })

# Initialize lists
combined_elements = {
    'exp_wavelengths': [],
    'raw_intensities': [],
    'ref_elements': [],
    'ref_wavelengths': [],
    'ref_intensities': []
}

# Loop through elements and get top lines
for set_number in range(len(exp_wavelength_matches_lists)):
    elements = {
        'exp_wavelengths': exp_wavelength_matches_lists[set_number],
        'raw_intensities': raw_intensities_lists[set_number],
        'ref_elements': ref_element_matches_lists[set_number],
        'ref_wavelengths': ref_wavelength_matches_lists[set_number],
        'ref_intensities': ref_intensity_matches_lists[set_number]
    }
    top_elements = get_top_elements(elements, top_lines_count)
    for i, el in enumerate(combined_elements.keys()):
        combined_elements[el].extend(top_elements[i])

# Create DataFrame and sort
combined_table = create_dataframe(*combined_elements.values())
combined_table = combined_table.sort_values(['Elemental Assignment', 'Intensity, Measured (Counts)'], ascending=[True, False])

# Display and save as Excel
display(combined_table)
new_analysis_book = pd.DataFrame(combined_table)
new_analysis_book.to_excel(final_file_name[0:len(final_file_name)-12]+'_elemental_analysis_summary.xlsx')

Next, we will generate a graph that labels the peaks above on our spectrum. It should match the table shown above perfectly!

In [ ]:
# REQUIRED
# PRODUCING ANNOTATED GRAPHS FOR REPORT

colors = px.colors.qualitative.G10

if len(ref_wavelength_matches_lists) > 10:
  colors = colors = px.colors.qualitative.Dark24

def closest_value(input_list, input_value):

  difference = lambda input_list : abs(input_list - input_value)

  res = min(input_list, key=difference)

  return res

# Making the simplfied annotated graph.
fig = go.Figure()
widval = 0.3
# Add scatter trace for line
fig.add_trace(go.Scatter(
    x=df["Wavelength (nm)"], y=df["Intensity (au)"],
    mode="lines",
    name = "Raw data",
    line=dict(color='black', width=0.4)
))

for set_number in range(len(ref_wavelength_matches_lists)):

    exp_wavelength_matches_list = [x for _, x in sorted(zip(raw_intensities_lists[set_number], exp_wavelength_matches_lists[set_number]), reverse=True)][0:top_lines_count]
    raw_intensities_list = [x for _, x in sorted(zip(raw_intensities_lists[set_number], raw_intensities_lists[set_number]), reverse=True)][0:top_lines_count]
    ref_element_matches_list = [x for _, x in sorted(zip(raw_intensities_lists[set_number], ref_element_matches_lists[set_number]), reverse=True)][0:top_lines_count]
    ref_wavelength_matches_list = [x for _, x in sorted(zip(raw_intensities_lists[set_number], ref_wavelength_matches_lists[set_number]), reverse=True)][0:top_lines_count]
    ref_intensity_matches_list = [x for _, x in sorted(zip(raw_intensities_lists[set_number], ref_intensity_matches_lists[set_number]), reverse=True)][0:top_lines_count]

    for i in ref_wavelength_matches_list:
            fig.add_vrect(
            x0=i-widval, x1=i+widval,
            fillcolor=colors[set_number], opacity=0.2,
            layer="below", line_width=0, row=2, col=1)

        #find matched wavelength, #find wavelength range to search through, #find wavelength with max value
    x_vals = []
    y_vals= []
    for i in ref_wavelength_matches_list:
        val=closest_value(df["Wavelength (nm)"],i)
        index = list(df["Wavelength (nm)"]).index(val)
        candidate_list = [
            list(df["Intensity (au)"])[index-7],
            list(df["Intensity (au)"])[index-6],
            list(df["Intensity (au)"])[index-5],
            list(df["Intensity (au)"])[index-4],
            list(df["Intensity (au)"])[index-3],
            list(df["Intensity (au)"])[index-2],
            list(df["Intensity (au)"])[index-1],
            list(df["Intensity (au)"])[index],
            list(df["Intensity (au)"])[index+1],
            list(df["Intensity (au)"])[index+2],
            list(df["Intensity (au)"])[index+3],
            list(df["Intensity (au)"])[index+4],
            list(df["Intensity (au)"])[index+5],
            list(df["Intensity (au)"])[index+6],
            list(df["Intensity (au)"])[index+7]]
        #print("The three options for the max are: ", candidate_list)
        index_max_value = max(candidate_list)
        #print("The max of these candidates is: ", index_max_value)
        index_max = candidate_list.index(index_max_value)
        #print("The internal index  of this value is: ", index_max)

        x_val = list(df["Wavelength (nm)"])[index+index_max-7]
        y_val = list(df["Intensity (au)"])[index+index_max-7]
        x_vals.append(x_val)
        y_vals.append(y_val)


    fig.add_trace(go.Scatter(x=x_vals, y=y_vals,
        mode="markers+text",
        name=str(ref_element_matches_list[0]),
        #text=ref_element_matches_list,
        textfont=dict(
        color='black'),
        marker={"size":5,
        "color": colors[set_number]},
        textposition = "bottom center"
    ))


fig.update_layout(template = 'none', height = 400, width =1200)
fig.update_layout(xaxis=dict(title='Wavelength (nm)', showgrid=False),
              yaxis=dict(title='Intensity (Counts)', showgrid=False)
)
fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
config = {'displayModeBar': True}
fig.show(config=config)


Feel free to use the graph and/or table in your report! If computer programming intimidates you, you are also free to ignore this code and do your own analysis from scratch using Excel and the NIST Lines database. Note that you must include at least one annotated graph from this code to show you completed this portion of the lab.

To analyze a new sample, click the Runtime tab above and select "Disconnect and Delete Runtime". Then, refresh the page. Now, run only the #REQUIRED cells.